In [1]:
from keras.layers import Input, Dense, RNN, LSTM, Concatenate, MaxPooling1D, Embedding
from keras.models import Model
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.models import model_from_json
import numpy as np
import pandas as pd
from time import time
import pickle
import json

/home/tpradhan/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
df1 = pd.read_hdf("jsons/yearwiseX1.h5",'main_dfX1')

In [6]:
df2 = pd.read_hdf("jsons/yearwiseX2.h5","main_dfX2")

In [7]:
df3 = pd.read_hdf("jsons/yearwisey.h5","main_dfy")

In [8]:
df3.shape

(7499042,)

In [9]:
df1 = df1.apply(lambda x: np.array(x))
df2 = df2.apply(lambda x: np.array(x))

In [10]:
input1 = np.asarray(df1.tolist())
input2 = np.asarray(df2.tolist())
input1 = input1.reshape(input1.shape[0] , 1, input1.shape[1] )
input2 = input2.reshape(input2.shape[0] , 1, input2.shape[1] )
main_output = df3.values

batch_size = 512
epochs = 8

In [11]:
main_output= main_output.reshape(main_output.shape[0],1)

In [17]:
main_input_1 = Input(shape=(1,50), name='main_input_1')
main_input_2 = Input(shape=(1,50), name='main_input_2')
lstm_out=LSTM(output_dim=50,activation='sigmoid',return_sequences=True)
lstm_out2=LSTM(output_dim=50,activation='sigmoid',return_sequences=False)
max_pooling=MaxPooling1D(pool_size=2,strides=1,padding='valid')
dense = Dense(1000,activation='relu')
dense2 = Dense(1000,activation='relu')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

/home/tpradhan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="sigmoid", return_sequences=True, units=50)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tpradhan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="sigmoid", return_sequences=False, units=50)`
  after removing the cwd from sys.path.


In [18]:
lstm_out_1=lstm_out(main_input_1)
print(lstm_out_1.shape)
lstm_out_2=lstm_out(main_input_2)

lstm_out_1=lstm_out2(lstm_out_1)
print(lstm_out_1.shape)
lstm_out_2=lstm_out2(lstm_out_2)
# max_pooling_1=max_pooling(lstm_out_1)
# max_pooling_2=max_pooling(lstm_out_2)


concatenate_layer=Concatenate()([lstm_out_1,lstm_out_2])
dense_output1 = dense2(concatenate_layer)

dense_output2 = dense(dense_output1)
logistic_regression_output=Dense(1,activation='sigmoid',name='main_output')(dense_output2)

(?, ?, 50)
(?, 50)


In [19]:
model = Model(inputs=[main_input_1, main_input_2], outputs=[logistic_regression_output])

model.compile(optimizer='RMSprop',
              loss={'main_output': 'mean_squared_error'},
              metrics=['mse', 'mae', 'mape', 'cosine'])

checkpoint = ModelCheckpoint("checkpoint", monitor='val_acc', verbose=1, mode='max')

model.fit({'main_input_1': input1, 'main_input_2': input2},
          {'main_output': main_output},
          epochs=epochs, 
          verbose=1,
          batch_size=batch_size,
          callbacks=[checkpoint],
          validation_split=0.2)

Train on 5999233 samples, validate on 1499809 samples
Epoch 1/8
5999233/5999233 [==============================] - 247s 41us/step - loss: 5.9057e-05 - mean_squared_error: 5.9057e-05 - mean_absolute_error: 0.0025 - mean_absolute_percentage_error: 1027612.0353 - cosine_proximity: -0.2859 - val_loss: 3.2798e-06 - val_mean_squared_error: 3.2798e-06 - val_mean_absolute_error: 0.0010 - val_mean_absolute_percentage_error: 725121.6003 - val_cosine_proximity: -0.1108

Epoch 00001: saving model to checkpoint
Epoch 2/8
5999233/5999233 [==============================] - 243s 41us/step - loss: 3.7804e-05 - mean_squared_error: 3.7804e-05 - mean_absolute_error: 0.0024 - mean_absolute_percentage_error: 929250.5853 - cosine_proximity: -0.2859 - val_loss: 2.0175e-06 - val_mean_squared_error: 2.0175e-06 - val_mean_absolute_error: 6.5099e-04 - val_mean_absolute_percentage_error: 321662.5696 - val_cosine_proximity: -0.1107

Epoch 00002: saving model to checkpoint
Epoch 3/8
 486400/5999233 [=>..............

KeyboardInterrupt: 

In [95]:
input1.shape

(7499042,)

In [112]:
a=np.array([])
count=0
for i in input1:
    if count%10000==0:
        print(count/10000,end=' ')
    count+=1
    a=np.concatenate((a,i))

0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 

KeyboardInterrupt: 

In [115]:
a=np.ndarray.flatten(input1)

In [105]:
a=np.ndarray.flatten(input1)

In [114]:
type(input1[0][0])

numpy.float64

In [118]:
a.shape

(7499042,)

In [117]:
type(input1)

numpy.ndarray

In [124]:
a=np.array([np.array([1,2,3,4]),np.array([2,3,4,5]),[6,7,8,9]])

In [129]:
type(a[0][0])

numpy.int64

In [126]:
b=np.ndarray.flatten(a)

In [136]:
b[0]

1

In [132]:
type(input1[0][0])

numpy.float64

In [134]:
a=np.ndarray.flatten(input1)

In [6]:
input1.shape

(7499042,)

In [7]:
i=np.asarray(df['X1'].tolist())

In [8]:
i.shape

(7499042, 50)

In [22]:
input1.shape

(7499042, 50, 1)

In [19]:
input2.shape

(7499042, 50, 1)

In [20]:
main_output.shape

(7499042, 1, 1)

In [ ]:
1499809

In [2]:
df1 = pd.read_hdf("jsons/yearwiseAuthor1.h5",'main_dfAuthor1')

In [3]:
df2 = pd.read_hdf("jsons/yearwiseAuthor2.h5",'main_dfAuthor2')

In [4]:
a1=np.array(list(df1.iloc[5999234:]))

In [5]:
a2=np.array(list(df2.iloc[5999234:]))

In [99]:
author1=set(df1.iloc[5999234:])

In [100]:
author2=set(df2.iloc[5999234:])

In [101]:
len(list(author1 | author2))

465849

In [6]:
author1=author1|author2

In [7]:
author1=list(author1)

In [8]:
author1x=set(df1.iloc[:5999234])

In [9]:
author2x=set(df2.iloc[:5999234])

In [10]:
author1x=list(author1x|author2x)

In [11]:
new = list(np.setdiff1d(np.array(author1),np.array(author1x), assume_unique=True))

In [38]:
pickle.dump(new,open("pickles/new_authors.pkl","wb"))

In [21]:
new_dict={key:[] for key in new}

In [ ]:
print("-----Start-----")
count = 0
for author in author1:
    if count%1000:
        print(count//1000,end=' ')
    try:
        new_dict[author].append(a2[np.array(a1=='author')])
    except:
        pass
print("-----End-----")

-----Start-----


In [51]:
author2.index('Ana C. L. Sá')

1

In [13]:
len(author1)

465849

In [77]:
a=np.array(['1j','1j','2d'])
np.where(a=='1j')[0]

array([0, 1])

In [80]:
df1

0                                F.C. Williams
1                                     Hao Wang
2                                C. D. Morrill
3                            Nicholas M. Smith
4                           Franklin C. Brooks
5                                 D. Blackwell
6                           Franklin C. Brooks
7                                 D. Blackwell
8                                 D. Blackwell
9                              David E. Debeau
10                                Max H. Kraus
11                               R. D. Elbourn
12                            Leonard J. Craig
13                                  A. Charnes
14                                  A. Charnes
15                                 Donald Farr
16                                C. J. Savant
17                              Alan L. Leiner
18                                Hannah Rosén
19                                B. G. Farley
20                                H. N. Garber
21           

In [14]:
new_dict = pickle.load(open("pickles/new_dict.pkl","rb"))

In [23]:
new_dict

{'Maher Abdelrasoul': [['Mohammed S. Sayed', 'Victor Goulart']],
 'Melissa Marsh': [[]],
 'Masao Kamimura': [['Yoshiyuki Midorikawa',
   'WeiJen Lai',
   'Zuisei Kanno',
   'Motohiro Uo']],
 'Agnete Dyssegaard': [['Anders Ettrup',
   'Claus Svarer',
   'Brenda J. McMahon',
   'Sofi da Cunha-Bang',
   'Szabolcs Lehel',
   'Kirsten Møller',
   'Melanie Ganz',
   'Vincent Beliveau',
   'Louise Møller Jørgensen',
   'Nic Gillings',
   'Gitte M. Knudsen']],
 'Tairan Liu': [[]],
 'Marta Torrens': [['Solène Grosdidier', 'Olga Valverde', 'Àlex Bravo']],
 'Tosapon Pankumhang': [[]],
 'Orsolya Valkó': [['P. Burai', 'Tamás Tomor']],
 'Zulqurnain Sabir': [[]],
 'V. Kalaiyarasi': [[]],
 'Kwstantinos Papadamou': [['Nicolas Tsapatsoulis',
   'Makis Stamatelatos',
   'Michael Sirivianos']],
 'Qingbo Wen': [[]],
 'Mirjam Walloth': [['Rolf Krause']],
 'Fariza Hanum': [['Shahaboddin Shamshirband']],
 'Sorour E. Amiri': [[]],
 'Jen-I Chen': [['Jonathan A. Berken',
   'Vincent L. Gracco',
   'Soles J',
   

In [4]:
final_df = pd.read_hdf("jsons/final_df_dl.h5",'main_df')

In [20]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [21]:
loaded_model.compile(optimizer='RMSprop',
              loss={'main_output': 'mean_squared_error'},
              metrics=['mse', 'mae', 'mape', 'cosine'])

In [24]:
final_df

,Author1,X1,Author2,X2,y
0,F.C. Williams,"[0.116421856, -0.206808, 0.13134108, -0.034457...",T. Kilburn,"[0.3004278, -0.08267075, 0.25601763, 0.1804201...",0.000000
1,Hao Wang,"[4.3905177, -2.7893012, 1.6123409, 4.6770196, ...",J. Barkley Rosser,"[0.068904676, -0.031223673, 0.34112173, 0.2862...",0.000005
2,Ernest Hirsch,"[-0.024433045, -0.2912776, 1.0327069, 0.267105...",Redouane Khemmar,"[-0.5317088, -0.22070304, -0.06412962, 0.34742...",0.000820
3,Ray Tsaih,"[0.6482518, -0.053281967, 0.5312325, 0.5471009...",Wan‐Ying Lin,"[0.44098806, 0.66608065, 0.023865296, 0.287066...",0.003284
4,Edward W. Chencinski,"[-0.42852956, 0.23266008, 1.1875072, 0.8594547...",Markus M. Helms,"[0.075874425, -0.3536332, 0.9419311, 1.8802533...",0.003532
5,Kook-Yeol Yoo,"[1.3711678, -0.45269772, 0.16066076, 0.7606619...",Si-Woong Lee,"[0.22568288, -0.6788419, 0.023284635, -0.03827...",0.000820
6,A. Roy,"[2.8160937, 0.1355735, -1.3092846, 2.6904602, ...",F. Luehring,"[0.11940511, -0.2517713, -0.08435344, 0.316567...",0.000820
7,Yuxin Jin,"[-0.09414977, 0.025712641, 0.17621773, 0.07654...",Yuxin Peng,"[3.8037653, -4.0674124, 0.1918826, -0.4633302,...",0.001010
8,B. G. Farley,"[-0.025311185, 0.024059597, 0.042996295, -0.04...",W. A. Clark,"[-0.0050616297, -0.009904964, 0.005541698, -0....",0.000027
9,J. Jamison,"[-0.13027838, 0.036430646, 0.17478262, -0.0269...",W. J. Giguere,"[-0.04036356, -0.011908867, 0.015232532, 0.136...",0.000056


In [28]:
final_df[final_df.Author1=='Maher Abdelrasoul'].iloc[0]

Author1                                    Maher Abdelrasoul
X1         [0.12605056, -0.04319915, 0.01397269, 0.242958...
Author2                                    Mohammed S. Sayed
X2         [0.38033482, -0.8674485, -1.3433352, 2.7679188...
y                                                 0.00316549
Name: 5755956, dtype: object

In [102]:
abc=final_df[final_df.Author1=='Maher Abdelrasoul']
abc=abc[abc.Author2=='Mohammed S. Sayed']

In [114]:
Author_old = list(set(final_df.Author1.iloc[:5999234])|set(final_df.Author2.iloc[:5999234]))

Author_new = list(set(final_df.Author1.iloc[5999234:])|set(final_df.Author2.iloc[5999234:]))

Author_new_test = list(np.setdiff1d(np.array(Author_new),np.array(Author_old), assume_unique=True))

In [115]:
len(Author_new_test)

200040

In [116]:
len(Author_new)

535916

In [105]:
abc

,Author1,X1,Author2,X2,y
5755956,Maher Abdelrasoul,"[0.12605056, -0.04319915, 0.01397269, 0.242958...",Mohammed S. Sayed,"[0.38033482, -0.8674485, -1.3433352, 2.7679188...",0.003165


In [113]:
X1=np.array(list(abc.X1))
print(X1)
print(type(X1))
print(type(X1[0]))
np.shape(X1)

[[ 0.12605056 -0.04319915  0.01397269  0.24295802  0.02947534 -0.3345107
  -0.12940373  0.06365595 -0.06639762 -0.4245414   0.25831914 -0.12509142
  -0.08152274  0.22129026 -0.10286635  0.29356283 -0.14952897  0.07826957
   0.02163844 -0.0444955  -0.11732419  0.02047464  0.06224946  0.20811945
  -0.0759363  -0.0785851   0.0800769  -0.07010776  0.01347854 -0.3198856
   0.14129934 -0.09591042  0.12689346  0.3930853   0.15438208 -0.03445013
   0.1606026   0.2527581  -0.03568909  0.06136347 -0.21754926 -0.00717893
  -0.02284821  0.01897223 -0.03888342  0.14789145  0.14555427  0.28887957
  -0.05768852  0.09089327]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


(1, 50)

In [9]:
Xeval1=final_df[final_df.Author1=='Maher Abdelrasoul'].iloc[0].X1
Xeval2=final_df[final_df.Author1=='Maher Abdelrasoul'].iloc[0].X2

In [11]:
print(Xeval1)

[0.12605056, -0.04319915, 0.01397269, 0.24295802, 0.029475342, -0.3345107, -0.12940373, 0.06365595, -0.06639762, -0.4245414, 0.25831914, -0.12509142, -0.08152274, 0.22129026, -0.10286635, 0.29356283, -0.14952897, 0.07826957, 0.021638438, -0.044495504, -0.11732419, 0.020474637, 0.062249463, 0.20811945, -0.0759363, -0.0785851, 0.080076896, -0.07010776, 0.013478541, -0.3198856, 0.14129934, -0.09591042, 0.12689346, 0.3930853, 0.15438208, -0.03445013, 0.1606026, 0.2527581, -0.035689086, 0.061363466, -0.21754926, -0.0071789334, -0.022848206, 0.018972233, -0.038883418, 0.14789145, 0.14555427, 0.28887957, -0.05768852, 0.09089327]


In [16]:
Xeval1=np.array(Xeval1)
Xeval2=np.array(Xeval2)

In [17]:
print(Xeval1)

[ 0.12605056 -0.04319915  0.01397269  0.24295802  0.02947534 -0.3345107
 -0.12940373  0.06365595 -0.06639762 -0.4245414   0.25831914 -0.12509142
 -0.08152274  0.22129026 -0.10286635  0.29356283 -0.14952897  0.07826957
  0.02163844 -0.0444955  -0.11732419  0.02047464  0.06224946  0.20811945
 -0.0759363  -0.0785851   0.0800769  -0.07010776  0.01347854 -0.3198856
  0.14129934 -0.09591042  0.12689346  0.3930853   0.15438208 -0.03445013
  0.1606026   0.2527581  -0.03568909  0.06136347 -0.21754926 -0.00717893
 -0.02284821  0.01897223 -0.03888342  0.14789145  0.14555427  0.28887957
 -0.05768852  0.09089327]


In [14]:
X1=np.array([])
X2=np.array([])

In [18]:
X1

array([], dtype=float64)

In [8]:
X1=np.append([X1],[Xeval1],axis=1)

In [52]:
X2=np.append([X2],[Xeval2],axis=1)

In [54]:
X1=X1.reshape(1,50,1)
X2=X2.reshape(1,50,1)

In [36]:
Xeval1 = np.asarray(Xeval1.tolist())
Xeval2 = np.asarray(Xeval2.tolist())
Xeval1 = Xeval1.reshape(Xeval1.shape[0] , Xeval1.shape[1] , 1)
Xeval2 = Xeval2.reshape(Xeval2.shape[0] , Xeval2.shape[1] , 1)

IndexError: tuple index out of range

In [55]:
score = loaded_model.predict([X1,X2], verbose=1)

1/1 [==============================] - 4s 4s/step


In [120]:
score[0][0][0]

0.00054408365

In [66]:
col = json.load(open("jsons/Author_newcollab_test.json"))
col_tr = json.load(open("jsons/Author_collab_list_test1.json"))

In [62]:
data = json.load(open("jsons/Author_numnewcollab_test.json"))

In [73]:
df=pd.read_json("jsons/dblp_0_train.json")

In [80]:
data[0]

['Wei Wang', 247]

In [89]:
len(col['Wei Wang'])

247

In [91]:
len(col)

4545

In [92]:
len(col_tr)

49980

In [93]:
len(data)

4545

In [132]:
print(col['Rabab K. Ward'])

['Li Deng', 'Stephen Lin', 'Xiang Chen', 'Jianfeng Gao', 'Dong Xu', 'Yu Liu']


In [137]:
print(col_tr['Rabab K. Ward'])

['Jianshu Chen', 'Z. Jane Wang', 'Xiaodong He', 'Li Deng', 'Angshul Majumdar', 'Xun Chen', 'Stephen Lin', 'Xiang Chen', 'Hu Peng', 'Kunal Narayan Chaudhury', 'Yu Liu', 'Amr Mohamed', 'Jianfeng Gao', 'Dong Xu', 'Huazhu Fu', 'Panos Nasiopoulos', 'Hassan Mansour']


In [134]:
print(data)

[['Wei Wang', 247], ['Yang Liu', 180], ['Yang Yang', 170], ['Wei Zhang', 169], ['Wei Li', 168], ['Lei Zhang', 165], ['Wei Liu', 165], ['Lei Wang', 165], ['Jun Wang', 161], ['Xin Wang', 154], ['Wei Chen', 153], ['Yu Zhang', 150], ['Yang Li', 143], ['Yan Zhang', 142], ['Yu Wang', 141], ['Jun Li', 138], ['Jun Zhang', 136], ['Athanasios V. Vasilakos', 133], ['Xin Li', 129], ['Jun Liu', 128], ['Peng Zhang', 128], ['Jing Wang', 127], ['Tao Wang', 126], ['Jie Zhang', 125], ['Jian Zhang', 124], ['Peng Li', 123], ['Jing Zhang', 122], ['Yan Wang', 121], ['Fan Zhang', 120], ['Hao Wang', 118], ['Jing Li', 116], ['Hao Zhang', 115], ['Yan Li', 114], ['Bo Li', 113], ['Peng Wang', 113], ['Yi Wang', 112], ['Hui Li', 111], ['Rui Zhang', 110], ['Tao Zhang', 110], ['Jie Chen', 109], ['Xuelong Li', 109], ['Gang Wang', 109], ['Yang Wang', 107], ['Jie Li', 106], ['Jian Wang', 106], ['Jian Li', 105], ['Li Zhang', 103], ['Li Wang', 102], ['Yi Yang', 101], ['Jing Liu', 101], ['Dong Wang', 101], ['Bin Li', 100],

In [139]:
for i in col.keys():
    if i not in col_tr.keys():
        print("fhbdb")